# Simple Pipeline
In this example we will demonstrate how to create a simple pipeline using the Alpaca data bundle.

#### Loading the data bundle

In [1]:
import os


os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '.zipline')

os.listdir(os.environ['ZIPLINE_ROOT'])
import zipline
from zipline.data import bundles

bundle_name = 'alpaca_api'
bundle_data = bundles.load(bundle_name)
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.data.data_portal import DataPortal

import pandas as pd

# Set the dataloader
pricing_loader = USEquityPricingLoader.without_fx(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)


# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader

# Set the trading calendar
trading_calendar = get_calendar('NYSE')

start_date = pd.Timestamp('2019-07-05', tz='utc')
end_date = pd.Timestamp('2020-11-13', tz='utc')

# Create a data portal
data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = start_date,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)

## Creating the Pipeline Engine

In [2]:
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.engine import SimplePipelineEngine

# Create a Pipeline engine
engine = SimplePipelineEngine(get_loader = choose_loader,
                              asset_finder = bundle_data.asset_finder)

## Create the Pipeline

In [26]:
from zipline.pipeline.domain import US_EQUITIES
from zipline.pipeline.factors import AverageDollarVolume
from zipline.pipeline import Pipeline

# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length = 5).top(10)

# Create an empty Pipeline with the given screen
pipeline = Pipeline(screen = universe, domain=US_EQUITIES)
pipeline.add(AverageDollarVolume(window_length = 5), "Dollar Volume")

## Run the Pipeline

In [27]:
# Set the start and end dates
start_date = pd.Timestamp('2020-10-29', tz = 'utc')
end_date = pd.Timestamp('2020-11-3', tz = 'utc')

# Run our pipeline for the given start and end dates
pipeline_output = engine.run_pipeline(pipeline, start_date, end_date)

pipeline_output.head()

Dollar Volume
2020-10-29 00:00:00+00:00 Equity(32 [AMZN])    1.308611e+10
                          Equity(230 [AMD])    5.501710e+09
                          Equity(394 [FSLY])   1.691980e+09
                          Equity(672 [BA])     4.316417e+09
                          Equity(908 [C])      1.727939e+09